In [1]:
from Acesso import Login
from Query import Query
from Email import Email
from RemoverArquivo import Remover
from Tempo import DataHora
import os
import pandas as pd
from glob import glob

s=Login()

sql=Query(s.usuario,s.senha,s.database,s.server)

pd.set_option('display.max_columns',None)

In [2]:
querys={

    'Cliente':

    """

    SELECT * FROM netfeira.vw_cliente
    
    """,

    'Vendas':

    """
    
    DECLARE @BASE DATETIME,@DTFIM DATETIME,@DTINICIO DATETIME,@DIAS SMALLINT

    SET @BASE=CONVERT(DATETIME,CAST(GETDATE() AS DATE),101)

    SET @DIAS=DAY(@BASE)*-1

    SET @DTFIM=DATEADD(DAY,@DIAS,@BASE)

    SET @DTINICIO=CONCAT(YEAR(@DTFIM),'-',MONTH(@DTFIM),'-01')

    SELECT * FROM netfeira.vw_targetestatico
    WHERE [Data de Faturamento] BETWEEN @DTINICIO AND @DTFIM
    AND [Tipo de Operação]='VENDAS'
    ORDER BY [Data de Faturamento]    
    
    """,
    
    'Listagem':
    
    """
    
    DECLARE @BASE DATETIME,@DTFIM DATETIME,@DTINICIO DATETIME,@DIAS SMALLINT

    SET @BASE=CONVERT(DATETIME,CAST(GETDATE() AS DATE),101)

    SET @DIAS=DAY(@BASE)*-1

    SET @DTFIM=DATEADD(DAY,@DIAS,@BASE)

    SET @DTINICIO=CONCAT(YEAR(@DTFIM),'-',MONTH(@DTFIM),'-01');

    SELECT * FROM (
    SELECT [ID Cliente],[ID Vendedor],[Pedido],[Data de Faturamento],SUM([Total Geral]) AS 'Total Geral'
    FROM netfeira.vw_targetestatico
    WHERE [Data de Faturamento]<=@DTFIM
    AND [Tipo de Operação]='VENDAS'
    GROUP BY [ID Cliente],[ID Vendedor],[Pedido],[Data de Faturamento])base
    WHERE [Total Geral]>0 
    
    """,
    
    'Vendedor':
    
    """
    
    SELECT * FROM netfeira.vw_vendedor
    WHERE [Status do Vendedor]='ATIVO'    
    
    """,
    
    'Supervisor':
    
    """
    
    SELECT * FROM netfeira.vw_supervisor
    
    """

}

In [3]:
tabelas_df=sql.CriarTabela(kwargs=querys)

In [4]:
tabelas_df['Cliente'].columns

Index(['ID Cliente', 'CNPJ', 'CNPJ Caracter', 'Razão Social', 'Nome Fantasia',
       'Tipo de Cliente', 'Status do Cliente', 'ID Segmento', 'Matriz',
       'Crédito', 'Data de Cadastro', 'Primeira Compra', 'Última Compra',
       'Dias Compra', 'Tabela', 'Condição de Pagto', 'Prazo Pagto',
       'Pagamento', 'CEP', 'Endereço', 'Bairro', 'Município', 'Numero',
       'Complemento', 'Região', 'UF', 'DDD', 'Contato', 'Limite de Crédito',
       'Principal'],
      dtype='object')

In [5]:
tabelas_df['Vendas'].columns

Index(['Data de Emissão', 'Data de Faturamento', 'Pedido', 'Nfe', 'ID Empresa',
       'ID Cliente', 'ID Vendedor', 'Tipo de Pedido', 'Tipo de Operação',
       'Tabelas', 'SKU', 'Qtde', 'Unid. VDA', 'Qtde VDA', 'Valor VDA',
       'Total Geral', 'Total Venda', 'Comissão R$', 'Margem Bruta R$',
       'Cad Vendedor', 'Situação', 'Peso Bruto KG', 'Peso Líquido KG'],
      dtype='object')

In [6]:
tabelas_df['Vendedor'].columns

Index(['ID Vendedor', 'Vendedor', 'Nome Resumido', 'ID Equipe', 'E-mail',
       'Categoria', 'Data de Cadastro', 'Status do Vendedor'],
      dtype='object')

In [7]:
tabelas_df['Supervisor'].columns

Index(['ID Equipe', 'Equipe', 'ID Sup', 'Supervisor', 'Email Sup',
       'ID Gerente', 'Gerente', 'Email Gerente'],
      dtype='object')

In [8]:
#Base dos vendedores

vendedor_df=pd.DataFrame()

vendedor_df=tabelas_df['Vendedor'].merge(tabelas_df['Supervisor'],on='ID Equipe',how='inner')[['ID Vendedor', 'Vendedor', 'Nome Resumido', 'Equipe', 'E-mail',
       'Categoria', 'Status do Vendedor', 'ID Sup', 'Supervisor', 'Email Sup',
       'ID Gerente', 'Gerente', 'Email Gerente']]

vendedor_df=vendedor_df.loc[~vendedor_df['Equipe'].str.contains('120')]

In [9]:
vendedor_df.columns

Index(['ID Vendedor', 'Vendedor', 'Nome Resumido', 'Equipe', 'E-mail',
       'Categoria', 'Status do Vendedor', 'ID Sup', 'Supervisor', 'Email Sup',
       'ID Gerente', 'Gerente', 'Email Gerente'],
      dtype='object')

In [17]:
#Analise de Abertura

#1º consolidar a base de dados

vendas_df=pd.DataFrame()

temp_df=pd.DataFrame()

cliente_df=pd.DataFrame()

abertura_df=pd.DataFrame()

vendas_df=tabelas_df['Vendas'].merge(tabelas_df['Cliente'],on='ID Cliente',how='inner')[['Data de Emissão', 'Data de Faturamento', 'Pedido', 'Nfe', 'ID Empresa',
       'ID Cliente','Data de Cadastro', 'ID Vendedor', 'Tipo de Pedido', 'Tipo de Operação',
       'Tabelas', 'SKU', 'Qtde', 'Unid. VDA', 'Qtde VDA', 'Valor VDA',
       'Total Geral', 'Total Venda', 'Comissão R$', 'Margem Bruta R$',
       'Cad Vendedor', 'Situação', 'Peso Bruto KG', 'Peso Líquido KG']]

vendas_df['Positivado']=vendas_df.apply(lambda info: 1 if info['Data de Emissão']==info['Data de Cadastro'] else 0,axis=1)

dt_max=vendas_df['Data de Faturamento'].max()

temp_df=vendas_df[['ID Cliente','Total Geral']].loc[(vendas_df['Data de Emissão'].dt.year==dt_max.year)&(vendas_df['Data de Emissão'].dt.month==dt_max.month)].groupby(['ID Cliente'],as_index=False).sum()

codigos=temp_df['ID Cliente'].loc[temp_df['Total Geral']<=0].unique().tolist()

vendas_df=vendas_df.loc[~vendas_df['ID Cliente'].isin(codigos)]

vendas_df=vendas_df.loc[(vendas_df['Data de Emissão'].dt.year==dt_max.year)&(vendas_df['Data de Emissão'].dt.month==dt_max.month)]

vendas_df=vendas_df.loc[vendas_df['Positivado']==1].groupby(['ID Cliente','ID Vendedor'],as_index=False).agg({'Total Geral':'sum','Data de Emissão':'min','Pedido':'min'})

vendas_df=vendas_df.loc[vendas_df['Total Geral']>0]

cliente_df=tabelas_df['Cliente'].drop(columns=tabelas_df['Cliente'].columns[-1])

cliente_df=cliente_df.merge(vendas_df,on='ID Cliente',how='inner')

abertura_df=cliente_df.merge(vendedor_df,on='ID Vendedor',how='inner')[['ID Cliente', 'CNPJ', 'CNPJ Caracter', 'Razão Social', 'Nome Fantasia',
       'Tipo de Cliente', 'Status do Cliente', 'ID Segmento', 'Matriz',
       'Crédito', 'Data de Cadastro', 'Primeira Compra', 'Última Compra',
       'Dias Compra', 'Tabela', 'Condição de Pagto', 'Prazo Pagto',
       'Pagamento', 'CEP', 'Endereço', 'Bairro', 'Município', 'Numero',
       'Complemento', 'Região', 'UF', 'DDD', 'Contato', 'Limite de Crédito',
       'ID Vendedor','Vendedor', 'Nome Resumido', 'Equipe', 'E-mail',
       'Categoria', 'Status do Vendedor', 'ID Sup', 'Supervisor', 'Email Sup',
       'ID Gerente', 'Gerente', 'Email Gerente','Total Geral', 'Data de Emissão', 'Pedido']]

abertura_df['Arquivo']='ABERTURA'

abertura_df

,ID Cliente,CNPJ,CNPJ Caracter,Razão Social,Nome Fantasia,Tipo de Cliente,Status do Cliente,ID Segmento,Matriz,Crédito,Data de Cadastro,Primeira Compra,Última Compra,Dias Compra,Tabela,Condição de Pagto,Prazo Pagto,Pagamento,CEP,Endereço,Bairro,Município,Numero,Complemento,Região,UF,DDD,Contato,Limite de Crédito,ID Vendedor,Vendedor,Nome Resumido,Equipe,E-mail,Categoria,Status do Vendedor,ID Sup,Supervisor,Email Sup,ID Gerente,Gerente,Email Gerente,Total Geral,Data de Emissão,Pedido,Arquivo
0,28111,08359422000402,14,SUPERMERCADO SEMAR DE CEZAR DE SOUZA LTDA,SEMAR LJ22,J,ATIVO,0001,SEMAR,NORMAL,2022-07-01,NaT,2022-07-08,35,TABELA DE PREÇO,BOLETO BANCARIO 30 DIAS,30.0,BANCO,8750600.0,"RUA JOSÉ GALLUCCI, 281",JUNDIAPEBA,MOGI DAS CRUZES,281,,MOGI,SP,11,47273000.0,475.11,REGIS,REGIS SAMPAIO,REGIS SAMPAIO,EQUIPE 2 (REP),,PJ,ATIVO,ULISSES,ULISSES BACCI,ULISSES@DEMARCHIBRASIL.COM.BR,ROGERIOF,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,2622.68,2022-07-01,409763,ABERTURA
1,28112,00245454000637,14,J.G.G. SUPERMERCADOS LTDA,SEMAR LJ9,J,ATIVO,0001,SEMAR,NORMAL,2022-07-01,NaT,2022-07-08,35,TABELA DE PREÇO,BOLETO BANCARIO 30 DIAS,30.0,BANCO,8620000.0,RODOVIA ÍNDIO TIBIRIÇÁ - DE 6001/6002 A 11899/...,IPELÂNDIA,SUZANO,4600,,SUZANO,SP,11,47486240.0,332.88,REGIS,REGIS SAMPAIO,REGIS SAMPAIO,EQUIPE 2 (REP),,PJ,ATIVO,ULISSES,ULISSES BACCI,ULISSES@DEMARCHIBRASIL.COM.BR,ROGERIOF,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,1624.76,2022-07-01,409767,ABERTURA
2,28120,04063469000554,14,COMERCIAL KEYPAR REPRESENTAÇÕES E SUPERMERCADO...,SUPERMERCADO SEMAR,J,ATIVO,0001,SEMAR,NORMAL,2022-07-01,NaT,2022-07-08,35,TABELA DE PREÇO,BOLETO BANCARIO 30 DIAS,30.0,BANCO,7401070.0,AVENIDA ARMANDO COLANGELO - ATÉ 899 - LADO ÍMP...,ARUJÁ CENTER VILLE,ARUJÁ,173,,ARUJÁ,SP,11,3503901.0,408.55,REGIS,REGIS SAMPAIO,REGIS SAMPAIO,EQUIPE 2 (REP),,PJ,ATIVO,ULISSES,ULISSES BACCI,ULISSES@DEMARCHIBRASIL.COM.BR,ROGERIOF,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,1339.64,2022-07-01,409836,ABERTURA
3,28113,06337107845,11,LUCIA HELENA RODRIGUES BONINI,LUCIA HELENA RODRIGU,F,ATIVO,0036,E-COMMERCE,NORMAL,2022-07-01,NaT,2022-07-02,41,TABELA CONSUMIDOR FINAL,A VISTA,1.0,DINHEIRO,4305050.0,"RUA GALEAZZO ALESSI, 70 - 141",VILA MONTE ALEGRE,SÃO PAULO,70,141,JABAQUARA,SP,11,981946860.0,300.00,ECOMMER,E-COMMERCE,E-COMMERCE,E-COMMERCE,PEDIDOS@DEMARCHIBRASIL.COM.BR,B2C,ATIVO,ROGERIOF,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,ROGERIOF,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,235.71,2022-07-01,409842,ABERTURA
4,28124,28744920865,11,FABIO BARBAGLI,FABIO BARBAGLI,F,ATIVO,0036,None,NORMAL,2022-07-01,NaT,2022-07-02,41,TABELA CONSUMIDOR FINAL,A VISTA,1.0,DINHEIRO,4567002.0,"RUA ARIZONA, 664 - APT 142",CIDADE MONÇÕES,SÃO PAULO,664,APT 142,ITAIM BIBI,SP,11,994598805.0,200.00,ECOMMER,E-COMMERCE,E-COMMERCE,E-COMMERCE,PEDIDOS@DEMARCHIBRASIL.COM.BR,B2C,ATIVO,ROGERIOF,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,ROGERIOF,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,157.90,2022-07-01,409862,ABERTURA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,28254,28618982000107,14,BM SUPERMERCADO LTDA,MERCADO J PRIMO,J,ATIVO,0037,None,NORMAL,2022-07-21,NaT,2022-07-22,21,TABELA DE PREÇO,BOLETO BANCARIO 07 DIAS,7.0,BANCO,7152390.0,"RUA MARIA ELIZA SILVA, 47",JARDIM BANANAL,GUARULHOS,47,,GUARULHOS,SP,11,986340035.0,500.00,FONTOLAN,CELSO FONTOLAN MOLINA,CELSO MOLINA,EQUIPE 3 (CLT),CELSO.FONTOLAN@DEMARCHISAOPAULO.COM.BR,CLT,ATIVO,ULISSES,ULISSES BACCI,ULISSES@DEMARCHIBRASIL.COM.BR,ROGERIOF,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,216.10,2022-07-21,414555,ABERTURA
167,28259,06047087011920,14,REDE D'OR SAO LUIZ S.A.,MATERNIDADE SAO LUIZ,J,ATIVO,0013,HOSP SAO LUIZ,RESTRIÇÃO INTERNA,2022-07-21,NaT,2022-07-22,21,TABELA DE PREÇO,BOLETO BANCARIO 30 DIAS,30.0,BANCO,4552050.0,"RUA HELENA, 29",VILA OLÍMPIA,SÃO

In [11]:
def Dias(info):
    
    res=info['Última Compra']-info['Penúltima Compra']
    
    res=str(res)
    
    lista=[l for l in res.split()]
    
    res=int(lista[0])
    
    return res
    
    pass

In [23]:
#Analise de reativação

vendas_df=pd.DataFrame()

temp_df=pd.DataFrame()

cliente_df=pd.DataFrame()

codigos=tabelas_df['Listagem']['ID Cliente'].unique().tolist()

for codigo in codigos:
    
    vendas_df=tabelas_df['Listagem']
    
    dt_ult=vendas_df['Data de Faturamento'].loc[vendas_df['ID Cliente']==codigo].max()
    
    dt_penul=vendas_df['Data de Faturamento'].loc[(vendas_df['ID Cliente']==codigo)&(vendas_df['Data de Faturamento']<dt_ult)].max()
    
    vendas_df.loc[vendas_df['ID Cliente']==codigo,'Última Compra']=dt_ult
    
    vendas_df.loc[vendas_df['ID Cliente']==codigo,'Penúltima Compra']=dt_penul
           
    #break
    
    pass

df=pd.DataFrame()

df=vendas_df.loc[~vendas_df['Penúltima Compra'].isnull()]

df['Dias']=df.apply(Dias,axis=1)

df

C:\Users\eduardo\AppData\Local\Temp/ipykernel_10924/2362277736.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Dias']=df.apply(Dias,axis=1)


,ID Cliente,ID Vendedor,Pedido,Data de Faturamento,Total Geral,Última Compra,Penúltima Compra,Dias
0,6,LOJA,110493,2019-02-07,56.500,2019-11-13,2019-06-27,139
1,6,LOJA,156796,2019-06-27,262.080,2019-11-13,2019-06-27,139
2,6,LOJA,202581,2019-11-13,44.360,2019-11-13,2019-06-27,139
3,10,AVULSO,26614,2018-05-22,1238.402,2022-05-26,2022-03-28,59
4,10,AVULSO,43224,2018-07-17,41.400,2022-05-26,2022-03-28,59
...,...,...,...,...,...,...,...,...
378870,28255,MARCOSLI,416087,2022-07-29,118.150,2022-07-29,2022-07-22,7
378872,28257,RENATAAP,414568,2022-07-22,285.490,2022-07-29,2022-07-22,7
378873,28257,RENATAAP,416076,2022-07-29,347.030,2022-07-29,2022-07-22,7
378918,28300,BRUNA,416203,2022-07-29,391.620,2022-07-31,2022-07-29,2
